# Task 1

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from collections import Counter

PAD_TOKEN = "<PAD>"
UNK_TOKEN = "<UNK>"
PAD_TAG_IDX = -100 
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
NUM_EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 0.001


In [2]:
print("--- Task 1: Tải và Tiền xử lý Dữ liệu ---")

dataset = load_dataset("lhoestq/conll2003")
print(f"Bộ dữ liệu đã tải: {dataset}")

--- Task 1: Tải và Tiền xử lý Dữ liệu ---


d:\10. ky1nam4\NLP\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in E:\hf_cache\hub\datasets--lhoestq--conll2003. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, ins

Bộ dữ liệu đã tải: DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})


In [14]:
train_sentences = dataset["train"]["tokens"]
train_tags_numerical = dataset["train"]["ner_tags"] 
val_sentences = dataset["validation"]["tokens"]
val_tags_numerical = dataset["validation"]["ner_tags"]
test_sentences = dataset["test"]["tokens"] 
test_tags_numerical = dataset["test"]["ner_tags"]

tag_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
print(f"Danh sách tên nhãn NER: {tag_names}")

Danh sách tên nhãn NER: ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']


In [15]:
def convert_numerical_tags_to_strings(tags_numerical):
    tags_string = []
    for sent_tags in tags_numerical:
        tags_string.append([tag_names[tag_idx] for tag_idx in sent_tags])
    return tags_string

train_tags = convert_numerical_tags_to_strings(train_tags_numerical)
val_tags = convert_numerical_tags_to_strings(val_tags_numerical)

In [20]:
word_counts = Counter(word for sentence in train_sentences for word in sentence)
words = list(word_counts.keys())

word_to_ix = {PAD_TOKEN: 0, UNK_TOKEN: 1}
for word in words:
    if word not in word_to_ix:
        word_to_ix[word] = len(word_to_ix)
        
tag_to_ix = {}
for tag in tag_names:
    if tag not in tag_to_ix:
        tag_to_ix[tag] = len(tag_to_ix)
        
VOCAB_SIZE = len(word_to_ix)
OUTPUT_SIZE = len(tag_to_ix)
print(f"\nKích thước Từ điển Từ (VOCAB_SIZE): {VOCAB_SIZE}")
print(f"Kích thước Từ điển Nhãn (OUTPUT_SIZE): {OUTPUT_SIZE}")

PAD_WORD_IDX = word_to_ix[PAD_TOKEN]


Kích thước Từ điển Từ (VOCAB_SIZE): 23625
Kích thước Từ điển Nhãn (OUTPUT_SIZE): 9


# Task 2: Tạo PyTorch Dataset và DataLoader

In [21]:
class NERDataset(Dataset):
    def __init__(self, sentences, tags, word_to_ix, tag_to_ix):
        self.sentences = sentences
        self.tags = tags
        self.word_to_ix = word_to_ix
        self.tag_to_ix = tag_to_ix
        self.unk_idx = word_to_ix[UNK_TOKEN]

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        tags = self.tags[idx]
        sentence_indices = [self.word_to_ix.get(word, self.unk_idx) for word in sentence]
        tag_indices = [self.tag_to_ix[tag] for tag in tags]
        
        return torch.tensor(sentence_indices, dtype=torch.long), torch.tensor(tag_indices, dtype=torch.long)

def collate_fn(batch):
    sentences = [item[0] for item in batch]
    tags = [item[1] for item in batch]
    
    sentences_padded = nn.utils.rnn.pad_sequence(
        sentences, 
        batch_first=True, 
        padding_value=PAD_WORD_IDX
    )
    tags_padded = nn.utils.rnn.pad_sequence(
        tags, 
        batch_first=True, 
        padding_value=PAD_TAG_IDX
    )
    
    return sentences_padded, tags_padded

train_dataset = NERDataset(train_sentences, train_tags, word_to_ix, tag_to_ix)
val_dataset = NERDataset(val_sentences, val_tags, word_to_ix, tag_to_ix)

train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Task 3: Xây dựng Mô hình RNN 

In [22]:
class SimpleRNNForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_size, rnn_type='GRU'):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=PAD_WORD_IDX)
        if rnn_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        elif rnn_type == 'GRU':
            self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        else:
            self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_size)

    def forward(self, sentences):
        embedded = self.embedding(sentences)
        output, _ = self.rnn(embedded)
        output_linear = self.linear(output)
        return output_linear

model = SimpleRNNForTokenClassification(
    vocab_size=VOCAB_SIZE, 
    embedding_dim=EMBEDDING_DIM, 
    hidden_dim=HIDDEN_DIM, 
    output_size=OUTPUT_SIZE,
    rnn_type='GRU' 
)

# Task 4: Huấn luyện Mô hình

In [23]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = nn.CrossEntropyLoss(ignore_index=PAD_TAG_IDX) 
model.train() 

for epoch in range(NUM_EPOCHS):
    total_loss = 0
    for sentences_batch, tags_batch in train_dataloader:
        optimizer.zero_grad()
        predictions = model(sentences_batch)
        loss = loss_function(
            predictions.view(-1, OUTPUT_SIZE), 
            tags_batch.view(-1)
        )
        
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Loss: {avg_loss:.4f}")

Epoch 1/5, Loss: 0.6537
Epoch 2/5, Loss: 0.3485
Epoch 3/5, Loss: 0.2215
Epoch 4/5, Loss: 0.1474
Epoch 5/5, Loss: 0.0983


# Task 5: Đánh giá Mô hình

In [24]:
def evaluate(model, dataloader):
    model.eval()
    total_correct = 0
    total_tokens = 0
    
    with torch.no_grad():
        for sentences_batch, tags_batch in dataloader:
            
            predictions_raw = model(sentences_batch)
            predictions_index = torch.argmax(predictions_raw, dim=-1)
            mask = (tags_batch != PAD_TAG_IDX)
            correct_predictions = (predictions_index == tags_batch) & mask
            
            total_correct += correct_predictions.sum().item()
            total_tokens += mask.sum().item()
            
    accuracy = total_correct / total_tokens if total_tokens > 0 else 0
    model.train()
    
    return accuracy

val_accuracy = evaluate(model, val_dataloader)
print(f"Độ chính xác trên tập validation: {val_accuracy*100:.2f}%")

ix_to_tag = {v: k for k, v in tag_to_ix.items()}

def predict_sentence(sentence_str, model, word_to_ix, ix_to_tag):
    print(f"\nCâu: \"{sentence_str}\"")
    tokens = sentence_str.split()
    
    unk_idx = word_to_ix[UNK_TOKEN]
    token_indices = [word_to_ix.get(token, unk_idx) for token in tokens]
    
    input_tensor = torch.tensor(token_indices, dtype=torch.long).unsqueeze(0)
    
    model.eval()
    with torch.no_grad():
        predictions_raw = model(input_tensor)
        predictions_index = torch.argmax(predictions_raw, dim=-1).squeeze(0).tolist()
        
    model.train()
    predicted_tags = [ix_to_tag[idx] for idx in predictions_index]
    
    print("Dự đoán:")
    for word, tag in zip(tokens, predicted_tags):
        print(f"({word}, {tag})")

test_sentence = "VNU University is located in Hanoi"
predict_sentence(test_sentence, model, word_to_ix, ix_to_tag)

print("\nKẾT QUẢ THỰC HIỆN")
print(f"• Độ chính xác trên tập validation: {val_accuracy*100:.2f}%")
print(f"• Ví dụ dự đoán câu mới: Xem kết quả in ra cho câu \"{test_sentence}\"")

Độ chính xác trên tập validation: 93.14%

Câu: "VNU University is located in Hanoi"
Dự đoán:
(VNU, B-ORG)
(University, I-ORG)
(is, O)
(located, O)
(in, O)
(Hanoi, B-LOC)

KẾT QUẢ THỰC HIỆN
• Độ chính xác trên tập validation: 93.14%
• Ví dụ dự đoán câu mới: Xem kết quả in ra cho câu "VNU University is located in Hanoi"


# Báo Cáo Lab 5: Xây dựng Mô hình RNN cho Bài toán Nhận dạng Thực thể Tên (NER)

* **Mục tiêu:** Xây dựng mô hình Mạng Nơ-ron Hồi quy (GRU) để phân loại thực thể tên trên từng token, sử dụng bộ dữ liệu CoNLL 2003.

---

## 1. Quá trình Thực hiện (Tasks 1-5)

### Task 1: Tải và Tiền xử lý Dữ liệu

1.  [cite_start]**Tải Dữ liệu:** Sử dụng thư viện `datasets` để tải bộ dữ liệu **CoNLL 2003**[cite: 13, 14].
    * **Khắc phục Lỗi API:** Do lỗi `AttributeError` khi truy cập thuộc tính `.names` của metadata, danh sách tên nhãn NER đã được **định nghĩa thủ công** theo chuẩn IOB2 của CoNLL 2003:
        ```python
        tag_names = ['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']
        ```
2.  **Xây dựng Từ điển (Vocabulary):**
    * **Kích thước Từ điển Từ (VOCAB_SIZE):** 23625
    * **Kích thước Từ điển Nhãn (OUTPUT_SIZE):** 9

---

### Task 2: Tạo PyTorch Dataset và DataLoader

1.  [cite_start]**Lớp `NERDataset`:** Xử lý việc ánh xạ từ và nhãn sang các chỉ số số nguyên (`token_indices`, `tag_indices`)[cite: 44, 47, 48].
2.  [cite_start]**Hàm `collate_fn`:** Thực hiện đệm (padding) động cho các câu và nhãn trong mỗi batch bằng `torch.nn.utils.rnn.pad_sequence`[cite: 52].
    * [cite_start]**Padding cho nhãn:** Sử dụng chỉ số đặc biệt **`PAD_TAG_IDX = -100`** để các token đệm bị bỏ qua trong tính toán hàm lỗi[cite: 53].

---

### Task 3: Xây dựng Mô hình RNN

[cite_start]Mô hình **`SimpleRNNForTokenClassification`** được xây dựng với kiến trúc 3 lớp[cite: 56]:
1.  **`nn.Embedding`**: Lớp embedding từ.
2.  [cite_start]**`nn.GRU`**: Sử dụng GRU (thay cho RNN cơ bản) để xử lý chuỗi và trích xuất ngữ cảnh[cite: 58].
3.  **`nn.Linear`**: Ánh xạ output của RNN sang không gian nhãn.

---

### Task 4: Huấn luyện Mô hình

1.  [cite_start]**Loss Function:** `nn.CrossEntropyLoss` được sử dụng[cite: 65, 66].
2.  [cite_start]**Thiết lập `ignore_index`:** Tham số **`ignore_index`** được đặt bằng **`-100`** để đảm bảo hàm lỗi bỏ qua các vị trí padding khi tính toán[cite: 67, 68].
3.  [cite_start]**Optimizer:** `torch.optim.Adam` được sử dụng[cite: 65].
4.  Mô hình được huấn luyện trong **5 epochs**.

---

## 2. Kết quả Đánh giá (Task 5)

### Đánh giá Độ chính xác trên Tập Validation

[cite_start]Độ chính xác được tính trên các token **không phải padding** của tập validation[cite: 81, 82].

| Chỉ số | Giá trị |
| :--- | :--- |
| **Độ chính xác trên tập validation (Accuracy)** | **93.14%** |

### Ví dụ Dự đoán Câu mới

[cite_start]Đã chạy hàm `predict_sentence` với câu thử nghiệm mới để kiểm tra khả năng khái quát hóa của mô hình[cite: 87]:

| Câu gốc: | "VNU University is located in Hanoi" |
| :--- | :--- |
| VNU | B-ORG |
| University | I-ORG |
| is | O |
| located | O |
| in | O |
| Hanoi | B-LOC |

**Nhận xét:** Mô hình đã nhận diện chính xác "VNU University" là một thực thể Tổ chức (ORG) và "Hanoi" là một thực thể Địa điểm (LOC).

---

## 4. Nộp bài

**KẾT QUẢ THỰC HIỆN**

* **Độ chính xác trên tập validation:** **93.14%**
* **Ví dụ dự đoán câu mới:** Xem kết quả in ra cho câu "VNU University is located in Hanoi"